
<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# Random Forests and ExtraTrees


_Authors: Matt Brems (DC), Riley Dallas (AUS)_

---

## Random Forests
---

With bagged decision trees, we generate many different trees on pretty similar data. These trees are **strongly correlated** with one another. Because these trees are correlated with one another, they will have high variance. Looking at the variance of the average of two random variables $T_1$ and $T_2$:

$$
\begin{eqnarray*}
Var\left(\frac{T_1+T_2}{2}\right) &=& \frac{1}{4}\left[Var(T_1) + Var(T_2) + 2Cov(T_1,T_2)\right]
\end{eqnarray*}
$$

If $T_1$ and $T_2$ are highly correlated, then the variance will about as high as we'd see with individual decision trees. By "de-correlating" our trees from one another, we can drastically reduce the variance of our model.

That's the difference between bagged decision trees and random forests! We're going to do the same thing as before, but we're going to de-correlate our trees. This will reduce our variance (at the expense of a small increase in bias) and thus should greatly improve the overall performance of the final model.

So how do we "de-correlate" our trees?

Random forests differ from bagging decision trees in only one way: they use a modified tree learning algorithm that selects, at each split in the learning process, a **random subset of the features**. This process is sometimes called the *random subspace method*.

The reason for doing this is the correlation of the trees in an ordinary bootstrap sample: if one or a few features are very strong predictors for the response variable (target output), these features will be used in many/all of the bagged decision trees, causing them to become correlated. By selecting a random subset of features at each split, we counter this correlation between base trees, strengthening the overall model.

For a problem with $p$ features, it is typical to use:

- $\sqrt{p}$ (rounded down) features in each split for a classification problem.
- $p/3$ (rounded down) with a minimum node size of 5 as the default for a regression problem.

While this is a guideline, Hastie and Tibshirani (authors of Introduction to Statistical Learning and Elements of Statistical Learning) have suggested this as a good rule in the absence of some rationale to do something different.

Random forests, a step beyond bagged decision trees, are **very widely used** classifiers and regressors. They are relatively simple to use because they require very few parameters to set and they perform pretty well.
- It is quite common for interviewers to ask how a random forest is constructed or how it is superior to a single decision tree.

--- 

## Extremely Randomized Trees (ExtraTrees)
Adding one more step of randomization (and thus de-correlation) yields extremely randomized trees, or _ExtraTrees_. These are trained using bagging (sampling of observations) and the random subspace method (sampling of features), but an additional layer of randomness is introduced. Instead of computing the locally optimal feature/split combination (based on, e.g., information gain or the Gini impurity) for each feature under consideration, a random value is selected for the split. This value is selected from the feature's empirical range.

This further reduces the variance, but causes an increase in bias. If you're considering using ExtraTrees, you might consider this to be a hyperparameter you can tune. Build an ExtraTrees model and a Random Forest model, then compare their performance!

That's exactly what we'll do below.

## Import libraries
---

We'll need the following libraries for today's lecture:
- `pandas`
- `numpy`
- `GridSearchCV`, `train_test_split` and `cross_val_score` from `sklearn`'s `model_selection` module 
- `RandomForestClassifier` and `ExtraTreesClassifier` from `sklearn`'s `ensemble` module 

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

## Load Data
---

Load `train.csv` and `test.csv` from Kaggle into `DataFrames`.

In [3]:
train = pd.read_csv('./datasets/train.csv')

In [4]:
test = pd.read_csv('./datasets/test.csv')

## Data Cleaning: Drop the two rows with missing `Embarked` values from train
---

In [5]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [9]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [13]:
train.dropna(subset=['Embarked'], inplace=True)

In [15]:
train.shape

(889, 12)

## Data Cleaning: `Fare`
---

The test set has one row with a missing value for `Fare`. Fill it with the average `Fare` with everyone from the same `Pclass`. **Use the training set to calculate the average!**

In [17]:
test['Pclass'].unique()

array([3, 2, 1], dtype=int64)

In [19]:
test.loc[test['Fare'].isnull(), 'Pclass']

152    3
Name: Pclass, dtype: int64

In [21]:
mean_fare_3 = train[train['Pclass']==3]['Fare'].mean()

In [22]:
test['Fare'] = test['Fare'].fillna(mean_fare_3)

In [25]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64

## Data Cleaning: `Age`
---

Let's simply impute all missing ages to be **999**. 

**NOTE**: This is not a best practice. However, 
1. Since we haven't really covered imputation in depth
2. And the proper way would take too long to implement (thus detracting) from today's lecture
3. And since we're ensembling with Decision Trees

We'll do it this way as a matter of convenience.

In [26]:
train['Age'] = train.Age.fillna(999)

In [27]:
test['Age'] = test.Age.fillna(999)

## Feature Engineering: `Cabin`
---

Since there are so many missing values for `Cabin`, let's binarize that column as follows:
- 1 if there originally was a value for `Cabin`
- 0 if it was null

**Do this for both `train` and `test`**

In [28]:
train.Cabin.isnull().sum()

687

In [29]:
train.Cabin = train.Cabin.notnull().astype(int)

In [30]:
test.Cabin = test.Cabin.notnull().astype(int)

## Feature Engineering: Dummies
---

Dummy the `Sex` and `Embarked` columns. Be sure to set `drop_first=True`.

In [31]:
train = pd.get_dummies(train, columns=['Sex', 'Embarked'], drop_first=True)

In [33]:
test = pd.get_dummies(test, columns=['Sex', 'Embarked'], drop_first=True)

## Model Prep: Create `X` and `y` variables
---

Our features will be:
- `Pclass`
- `Age`
- `SibSp`
- `Parch`
- `Fare`
- `Cabin`
- `Sex_male`
- `Embarked_Q`
- `Embarked_S`

And our target will be `Survived`

In [34]:
features = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Sex_male', 'Embarked_Q', 'Embarked_S']
X = train[features]
y = train['Survived']

## Callenge: What is our baseline accuracy?
---

The baseline accuracy is the percentage of the majority class, regardless of whether it is 1 or 0. It serves as the benchmark for our model to beat.

In [35]:
y.mean()

0.38245219347581555

In [36]:
y.value_counts(normalize=True)

0    0.617548
1    0.382452
Name: Survived, dtype: float64

## Train/Test Split
---

I know it can be confusing having an `X_test` from our training data vs a test set from Kaggle. If you want, you can use `X_val`/`y_val` for what we normally call `X_test`/`y_test`.

In [39]:
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42, stratify=y)

## Model instantiation
---

Create an instance of `RandomForestClassifier` and `ExtraTreesClassifier`.

In [40]:
rf = RandomForestClassifier(n_estimators=100)

In [41]:
et = ExtraTreesClassifier(n_estimators=100)

## Model Evaluation
---

Which one has a higher `cross_val_score`?

In [42]:
cross_val_score(rf, X_train, y_train, cv=3).mean()

0.8048048048048049

In [43]:
cross_val_score(et, X_train, y_train, cv=3).mean()

0.7867867867867867

## Grid Search
---

They're both pretty close performance-wise. We could Grid Search over both, but for the sake of time we'll go with `RandomForestClassifier`.

In [44]:
rf_params = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 1, 2, 3, 4, 5],
}
gs = GridSearchCV(rf, param_grid=rf_params, cv=5, n_jobs=-1)
gs.fit(X_train, y_train)
print(gs.best_score_)
gs.best_params_

0.8153153153153153


{'max_depth': 5, 'n_estimators': 100}

## Kaggle Submission
---

Now that we've evaluated our model, let's submit our predictions to Kaggle.